#Load Dataset

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-11-10 21:46:23--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbfedd3d38fb849bf2ecb28f836.dl.dropboxusercontent.com/cd/0/inline/CeKJmA883ZllPhZL8_nPGwY2fNPOE-QeEns64ocS_GsrUmOGHAAJ8fuBQLznT5LnAzMtZgs-H-tqT1Lyrjg-7Gi5mOMGlwrD5HYYWky8WtxJhzFiM6XVwW9LgTD0rrAOh7k/file?dl=1# [following]
--2024-11-10 21:46:24--  https://ucbfedd3d38fb849bf2ecb28f836.dl.dropboxusercontent.com/cd/0/inline/CeKJmA883ZllPhZL8_nPGwY2fNPOE-QeEns64ocS_GsrUmOGHAAJ8fuBQLznT5LnAzMtZgs-H-tqT1Lyrjg-7Gi5mOMGlwrD5HYYWky8WtxJhzFiM6XVwW9LgTD0rrAOh7k/file?dl=1
Resolving ucbfedd3d38fb849bf2ecb28f836.dl.dropboxusercontent.com (ucbfedd3d38fb849bf2ecb28f836.dl.dropboxusercontent.com)... 162.125.5.15, 2620:100:601d:15::a27d

#Load HF model

In [2]:
from huggingface_hub import notebook_login

notebook_login()



In [3]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [5]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
def generate_prompt(data):
  question = data['question']
  ref = data['references']

  chat = [
      {"role": "user", "content": f"Answer the Question: {question} Considering this information: {ref}"},
    ]

  data['prompt'] = tokenizer.apply_chat_template(chat, tokenize=False)
  return data

#dataset['test'] = dataset['test'].map(generate_prompt)

In [ ]:
def generate_zeroshot_prompt(likes, dislikes):
  likes_str = "\n".join(likes)
  dislikes_str = "\n".join(dislikes)

  chat = [
    {"role": "user", "content": f"I've liked these movies: {likes_str}. I've disliked these movies: {dislikes_str}. Recommend a new movie for me to watch."}
  ]
  return tokenizer.apply_chat_template(chat, tokenize=False)

In [28]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = f"Pretend you are a movie recommender system. I will give you a conversation between a human and assistant. Based on the conversation, you reply me with 50 recommendations without extra sentences. Here is the conversation: {conversation}."
    chat = [
        {"role": "user", "content": content}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

In [25]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [22]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [23]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

In [29]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.01 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [31]:
prompts = []
ground_truth = []
for user_id in sampled_users:
  user_data = read_user_data(path_movie_data, user_id)
  conversation_id = user_data['Conversation'][0]['conversation_1']['conversation_id']
  prompt = generate_zeroshot(conversation_id, conversations)
  prompts.append(prompt)


In [32]:
answers = []
for prompt in prompts:
  ans = text_generator(prompt, max_new_tokens=30)
  answers.append(ans[0]['generated_text'])


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [35]:
for answer in answers:
  print(answer)


<s>[INST] Pretend you are a movie recommender system. I will give you a conversation between a human and assistant. Based on the conversation, you reply me with 50 recommendations without extra sentences. Here is the conversation: User: Hi there! Have you seen "Earth to Echo"? I was quite disappointed with it. The movie didn't live up to my expectations based on the trailers. The characters were unlikable, especially Tuck, played by "Astro." The acting was subpar, and the plot seemed poorly written. The government construction crew storyline was also confusing and unresolved.
Agent: I understand your disappointment with "Earth to Echo." It seems like the movie didn't live up to its potential. The lack of closure and inconsistent filming style were also drawbacks. However, if you're looking for a recommendation, I suggest watching "Tucker & Dale vs. Evil." It's a hilarious and original comedy with a well-written script and great performances.
User: That sounds interesting! I'll definite